In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
rand_state=1000

In [2]:
train_df=pd.read_csv(r"D:\python\Scripts\titanic-dataset\train.csv")
test_df=pd.read_csv(r"D:\python\Scripts\titanic-dataset\test.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
train_df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
train_df['Cabin'].ffill(inplace=True)
train_df['Cabin'].replace(np.nan,"P10",inplace=True)
train_df['Age'] = train_df["Age"].fillna(train_df.Age.mean())
train_df["Embarked"] = train_df["Embarked"].fillna("C")
train_df.Sex = train_df.Sex.replace(train_df.Sex.unique(),[0,1])
train_df.Embarked = train_df.Embarked.replace(train_df.Embarked.unique(),[0,1,2])
train_df.Cabin,_ = pd.factorize(train_df.Cabin)


In [6]:
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,0,22.000000,1,0,7.2500,0,0
1,1,1,1,38.000000,1,0,71.2833,1,1
2,1,3,1,26.000000,0,0,7.9250,1,0
3,1,1,1,35.000000,1,0,53.1000,2,0
4,0,3,0,35.000000,0,0,8.0500,2,0
...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.000000,0,0,13.0000,145,0
887,1,1,1,19.000000,0,0,30.0000,146,0
888,0,3,1,29.699118,1,2,23.4500,146,0
889,1,1,0,26.000000,0,0,30.0000,147,1


In [7]:
from sklearn.model_selection import train_test_split
y=train_df['Survived']
X=train_df.drop('Survived',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=rand_state)

In [8]:
import xgboost as xgb
import optuna
from sklearn.metrics import accuracy_score
def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print("Best Hyperparameters:", best_params)

final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
acc_test = accuracy_score(y_test, y_test_pred)

print("Accuracy on test Set:", acc_test)

[I 2024-01-05 20:58:30,369] A new study created in memory with name: no-name-c861cedb-8db7-445b-bf1f-80eecec8bf35
[I 2024-01-05 20:58:30,517] Trial 0 finished with value: 0.8208955223880597 and parameters: {'colsample_bytree': 0.9440743594623229, 'learning_rate': 0.014924442350680751, 'max_depth': 8, 'subsample': 0.5675931535205934, 'gamma': 0.6553229738676705, 'min_child_weight': 4, 'reg_alpha': 0.7379639585591123, 'reg_lambda': 0.25690098162929287, 'n_estimators': 84}. Best is trial 0 with value: 0.8208955223880597.
[I 2024-01-05 20:58:31,184] Trial 1 finished with value: 0.8246268656716418 and parameters: {'colsample_bytree': 0.8320862939575338, 'learning_rate': 0.13336879094639903, 'max_depth': 9, 'subsample': 0.8274201878754022, 'gamma': 0.013701184601774274, 'min_child_weight': 1, 'reg_alpha': 0.08721861230607608, 'reg_lambda': 0.9494090709963263, 'n_estimators': 394}. Best is trial 1 with value: 0.8246268656716418.
[I 2024-01-05 20:58:31,477] Trial 2 finished with value: 0.82835

Best Hyperparameters: {'colsample_bytree': 0.41970234269207324, 'learning_rate': 0.07592049464963309, 'max_depth': 5, 'subsample': 0.9355510989487663, 'gamma': 0.9709285447348966, 'min_child_weight': 2, 'reg_alpha': 0.012262675279657298, 'reg_lambda': 0.5602704156541791, 'n_estimators': 179}
Accuracy on test Set: 0.832089552238806


In [9]:
passenger = test_df['PassengerId']
test_df.drop(['PassengerId'],axis=1,inplace=True)
test_df.drop(['Name','Ticket'],axis=1,inplace=True)
passenger

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [10]:
test_df['Cabin'].ffill(inplace=True)
test_df['Cabin'].replace(np.nan,"Z10",inplace=True)
test_df.isnull().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Cabin        0
Embarked     0
dtype: int64

In [11]:
test_df['Fare'].replace(np.nan,test_df['Fare'].mean(),inplace=True)
test_df['Age'].replace(np.nan,test_df['Age'].mean(),inplace=True)
test_df.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [12]:
test_df.Cabin,_ = pd.factorize(test_df.Cabin)
test_df.Sex = test_df.Sex.replace(test_df.Sex.unique(),[0,1])
test_df.Embarked = test_df.Embarked.replace(test_df.Embarked.unique(),[0,1,2])
final_test = test_df.values

In [13]:
pred_test=final_model.predict(final_test)
final = pd.DataFrame(data=(passenger),columns=['PassengerId'])
final.loc[:,"Survived"]=pred_test


In [14]:
final.to_csv('gender_submission',index=False)

In [15]:
tab=pd.read_csv('./gender_submission')
tab.head(25)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0
